## Instructions

### This code is used determine the bone loss angle and the bone loss patterns using the mathematical approach.

### Directory structure

```plaintext
images/
    1.jpg
    2.jpg
    3.jpg
    ...
yolo_mask_outputs_denorm_lines/
    7px/
        test/
            1.jpg
            2.jpg
            ...
        train/
            3.jpg
            4.jpg
            ...
        val/
            5.jpg
            6.jpg
            ...
    8px/
    ...
teeth_masks/
    test/
        1.png
        2.png
        ...
    train/
        3.png
        4.png
        ...
    val/
        5.png
        6.png
        ...
keypoints_0707/
    test/
        1.json
        2.json
        ...
    train/
        3.json
        4.json
        ...
    val/
        5.json
        6.json
        ...
```
### Directory structure explanation
- `images/`: Contains the original images used for training and testing.
- `yolo_mask_outputs_denorm_lines/`: Contains the YOLO mask outputs with denormalized lines, organized by pixel size (e.g., `7px`, `8px`).
  - Each subdirectory (`test`, `train`, `val`) contains images corresponding to that dataset split.
- `teeth_masks/`: Contains the masks for teeth, organized by dataset split (`test`, `train`, `val`).
- `keypoints_0707/`: Contains JSON files with keypoints for each image, organized by dataset split (`test`, `train`, `val`).
            


# Display for verifying

In [ ]:
!rm -rf display

In [ ]:
import os
from PIL import Image, ImageDraw
import json

# Set the directory paths
images_dir = "images/"
bone_details_base_dir = "yolo_mask_outputs_denorm_lines/10px"
mask_details_base_dir = "teeth_masks"
keypoints_base_dir = "keypoints_0707"
output_base_dir = "display"

# Ensure the output directory exists
os.makedirs(output_base_dir, exist_ok=True)

# Directories to process
subdirs = ["test", "train", "val"]

# Process each subdirectory
for subdir in subdirs:
    bone_details_dir = os.path.join(bone_details_base_dir, subdir)
    mask_details_dir = os.path.join(mask_details_base_dir, subdir)
    keypoints_dir = os.path.join(keypoints_base_dir, subdir)
    output_dir = os.path.join(output_base_dir, subdir)

    # Create output subdirectory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Process each file in the subdirectory
    for filename in os.listdir(bone_details_dir):
        if filename.endswith(".txt"):
            image_name = os.path.splitext(filename)[0] + ".jpg"
            image_path = os.path.join(images_dir, image_name)
            bone_details_path = os.path.join(bone_details_dir, filename)
            mask_details_path = os.path.join(mask_details_dir, filename)
            keypoints_path = os.path.join(keypoints_dir, os.path.splitext(filename)[0] + ".json")

            # Check if image, mask file, and keypoints file exist
            if not os.path.exists(image_path):
                print(f"Image {image_name} does not exist in {images_dir}. Skipping.")
                continue
            if not os.path.exists(mask_details_path):
                print(f"Mask file {filename} does not exist in {mask_details_dir}. Skipping.")
                continue
            if not os.path.exists(keypoints_path):
                print(f"Keypoints file {filename} does not exist in {keypoints_dir}. Skipping.")
                continue

            # Load image
            image = Image.open(image_path)
            draw = ImageDraw.Draw(image)

            # Draw bone lines
            with open(bone_details_path, "r") as f:
                bone_details = f.readlines()
                for line in bone_details:
                    points = [float(p) for p in line.split(" ")]
                    points = [(points[i], points[i + 1]) for i in range(0, len(points), 2)]
                    draw.line(points, fill="red", width=2)

            # Draw masks
            with open(mask_details_path, "r") as f:
                mask_details = f.readlines()
                for line in mask_details:
                    points = [float(p) for p in line.split()]
                    points = [(points[i], points[i + 1]) for i in range(0, len(points), 2)]
                    draw.polygon(points, outline="blue", width=2)

            # Draw keypoints
            with open(keypoints_path, "r") as f:
                keypoints_data = json.load(f)
                for keypoints in keypoints_data["keypoints"]:
                    for i, point in enumerate(keypoints):
                        x, y = point[0], point[1]
                        if (x, y) != (0, 0):
                            color = "green" if i < 2 else "yellow" if i < 4 else "purple"
                            draw.ellipse([(x - 3, y - 3), (x + 3, y + 3)], outline=color, width=2)

            # Save the image with bone lines, masks, and keypoints
            output_path = os.path.join(output_dir, image_name)
            image.save(output_path)

print("Images with bone lines, masks, and keypoints saved in:", output_base_dir)


# get results with lines and numerical values

In [ ]:
!rm -rf angles

In [ ]:
import numpy as np
import math
import json

# Function to classify an image as mandibular or maxillary based on keypoints
def classify_image(keypoints):
    maxillary_count = 0
    mandibular_count = 0

    for keypoint_array in keypoints:
        cej_coords = [keypoint_array[0][1], keypoint_array[1][1]]
        apex_coords = []

        if len(keypoint_array) == 6:
            apex_coords = [keypoint_array[4][1], keypoint_array[5][1]]
        elif len(keypoint_array) == 5:
            apex_coords = [keypoint_array[4][1]]

        cej_coords = [x for x in cej_coords if x != 0]
        apex_coords = [x for x in apex_coords if x != 0]

        if len(cej_coords) == 0 or len(apex_coords) == 0:
            return "mandibular" # assume it's mandibular
        cej_above_apex = sum(cej_y < min(apex_coords) for cej_y in cej_coords)
        cej_below_apex = len(cej_coords) - cej_above_apex

        if cej_above_apex >= cej_below_apex:
            mandibular_count += 1
        else:
            maxillary_count += 1

    if maxillary_count >= mandibular_count:
        return "maxillary"
    else:
        return "mandibular"

# Function to check if a point is inside a polygon using the ray-casting algorithm
def point_in_polygon(point, polygon):
    x, y = point
    inside = False
    n = len(polygon)
    px, py = polygon[0]
    for i in range(1, n + 1):
        sx, sy = polygon[i % n]
        if min(py, sy) < y <= max(py, sy):
            if x <= max(px, sx):
                if py != sy:
                    xinters = (y - py) * (sx - px) / (sy - py) + px
                if px == sx or x <= xinters:
                    inside = not inside
        px, py = sx, sy
    return inside

# Function to remove points inside masks
def remove_points_inside_masks(bones, masks):
    cleaned_bones = []
    for bone in bones:
        cleaned_bone = []
        for point in bone:
            if not any(point_in_polygon(point, mask) for mask in masks):
                cleaned_bone.append(point)
        if cleaned_bone != []:
            cleaned_bones.append(cleaned_bone)
    return cleaned_bones

def distance(p1, p2):
    return math.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

def interpolate(p1, p2, num_points):
    return [
        (
            p1[0] + (p2[0] - p1[0]) * i / (num_points + 1),
            p1[1] + (p2[1] - p1[1]) * i / (num_points + 1)
        )
        for i in range(1, num_points + 1)
    ]


In [ ]:
import os
from PIL import Image, ImageDraw, ImageFont
import json
import math
import numpy as np

# Set constants
LINE_EXTENSION = 20
MASK_SEARCH_DISTANCE = 20
SKIP_THRESHOLD = 20
ANGULAR_ANGLE = 55

font = ImageFont.truetype(r'/usr/share/fonts/truetype/liberation/LiberationMono-Regular.ttf', 20)

# Set the directory paths
images_dir = "images/"
bone_details_base_dir = "yolo_mask_outputs_denorm_lines/10px"
mask_details_base_dir = "teeth_masks"
keypoints_base_dir = "keypoints_0707"
output_base_dir = "angles"

# Create output directory if it doesn't exist
os.makedirs(output_base_dir, exist_ok=True)

image_classification = {}
for subdir in ["test", "train", "val"]:
    keypoints_dir = os.path.join(keypoints_base_dir, subdir)
    for filename in os.listdir(keypoints_dir):
        if filename.endswith(".json"):
            keypoints_path = os.path.join(keypoints_dir, filename)
            with open(keypoints_path, 'r') as f:
                keypoints_data = json.load(f)
            image_classification[os.path.splitext(filename)[0] + '.jpg'] = classify_image(keypoints_data["keypoints"])

# Directories to process
subdirs = ["test", "train", "val"]

# Process each subdirectory
for subdir in subdirs:
    bone_details_dir = os.path.join(bone_details_base_dir, subdir)
    mask_details_dir = os.path.join(mask_details_base_dir, subdir)
    output_dir = os.path.join(output_base_dir, subdir)

    # Create output subdirectory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Process each file in the subdirectory
    for filename in os.listdir(bone_details_dir):
        if filename.endswith(".txt"):
            # if filename != "10.txt":
            #     continue
            print(filename)
            image_name = os.path.splitext(filename)[0] + ".jpg"
            image_path = os.path.join(images_dir, image_name)
            bone_details_path = os.path.join(bone_details_dir, filename)
            mask_details_path = os.path.join(mask_details_dir, filename)

            # Check if bone details or mask details are missing, skip if so
            if not (os.path.exists(bone_details_path) and os.path.exists(mask_details_path)):
                print(f"Skipping {filename} due to missing bone or mask details.")
                continue

            # Read bone lines
            bones = []
            with open(bone_details_path, "r") as f:
                bone_details = f.readlines()
                for line in bone_details:
                    points = [float(p) for p in line.split(" ")]
                    points = [(points[i], points[i + 1]) for i in range(0, len(points), 2)]
                    bones.append(points)

            # Read teeth masks
            masks = []
            with open(mask_details_path, "r") as f:
                mask_details = f.readlines()
                for line in mask_details:
                    points = [float(p) for p in line.split()]
                    points = [(points[i], points[i + 1]) for i in range(0, len(points), 2)]

                    resampled_points = []
                    for i in range(len(points)):
                        resampled_points.append(points[i])
                        next_point = points[(i + 1) % len(points)]  # Wrap around to the first point

                        if distance(points[i], next_point) > 10:
                            num_extra_points = int(distance(points[i], next_point) / 10)
                            resampled_points.extend(interpolate(points[i], next_point, num_extra_points))

                    masks.append(resampled_points)

            if len(bones) == 0 or len(masks) == 0:
                print(f"Skipping {filename} due to empty bone or mask details.")
                continue

            # Load the image
            image = Image.open(image_path)
            draw = ImageDraw.Draw(image)

            # Draw bone lines in blue
            for bone in bones:
                draw.line(bone, fill="blue", width=4)

            # Remove points inside masks
            bones = remove_points_inside_masks(bones, masks)

            # Draw masks in green
            for mask in masks:
                draw.polygon(mask, outline="green", width=2)

            # Process each bone
            for bone in bones:
                for end in [0, -1]:  # 0 for start point, -1 for end point
                    point = bone[end]

                    # Find the closest point on masks to the point of the bone line
                    closest_mask_point = min(
                        (p for mask in masks for p in mask),
                        key=lambda p: math.dist(point, p)
                    )

                    if math.dist(point, closest_mask_point) >= SKIP_THRESHOLD:
                        print("exceeds skip")
                        print(math.dist(point, closest_mask_point))
                        continue

                    # Find the mask and index of the closest point
                    for i, mask in enumerate(masks):
                        if closest_mask_point in mask:
                            closest_mask_index = i
                            closest_mask_point_index = mask.index(closest_mask_point)
                            break

                    # Get another point on mask
                    nearby_mask_point_index = (closest_mask_point_index + MASK_SEARCH_DISTANCE) % len(masks[closest_mask_index])
                    nearby_mask_point = masks[closest_mask_index][nearby_mask_point_index]

                    # Ensure that nearby_mask_point is in APEX side
                    if image_classification.get(image_name) == "mandibular":
                        if nearby_mask_point[1] < point[1]:
                            nearby_mask_point_index = (closest_mask_point_index - MASK_SEARCH_DISTANCE) % len(masks[closest_mask_index])
                            nearby_mask_point = masks[closest_mask_index][nearby_mask_point_index]
                    if image_classification.get(image_name) == "maxillary":
                        if nearby_mask_point[1] > point[1]:
                            nearby_mask_point_index = (closest_mask_point_index - MASK_SEARCH_DISTANCE) % len(masks[closest_mask_index])
                            nearby_mask_point = masks[closest_mask_index][nearby_mask_point_index]

                    # To extend the lines
                    start_mask_point = (
                        closest_mask_point[0] - LINE_EXTENSION * (closest_mask_point[0] - nearby_mask_point[0]),
                        closest_mask_point[1] - LINE_EXTENSION * (closest_mask_point[1] - nearby_mask_point[1])
                    )
                    end_mask_point = (
                        closest_mask_point[0] + LINE_EXTENSION * (closest_mask_point[0] - nearby_mask_point[0]),
                        closest_mask_point[1] + LINE_EXTENSION * (closest_mask_point[1] - nearby_mask_point[1])
                    )

                    # get mid point index of bone
                    mid_point_index = len(bone) // 3
                    POINTS_AWAY = mid_point_index

                    # Calculate vectors for bone and mask tangent lines
                    if end == 0:  # start point
                        bone_vector = np.array([bone[POINTS_AWAY][0] - point[0], bone[POINTS_AWAY][1] - point[1]])
                    else:  # end point
                        bone_vector = np.array([bone[-1-POINTS_AWAY][0] - point[0], bone[-1-POINTS_AWAY][1] - point[1]])
                    mask_vector = np.array([end_mask_point[0] - start_mask_point[0], end_mask_point[1] - start_mask_point[1]])

                    # Calculate dot product and magnitudes
                    dot_product = np.dot(bone_vector, mask_vector)
                    bone_magnitude = np.linalg.norm(bone_vector)
                    mask_magnitude = np.linalg.norm(mask_vector)

                    # Calculate angle in radians
                    angle_rad = np.arccos(dot_product / (bone_magnitude * mask_magnitude))

                    # Convert angle to degrees
                    angle_deg = np.degrees(angle_rad)

                    # Draw angle text on the image
                    angle_text = f"{angle_deg:.2f}°"
                    if angle_deg <= ANGULAR_ANGLE:
                        circle_center = ((point[0], point[1]))
                        circle_radius = 10
                        draw.ellipse((
                            circle_center[0] - circle_radius, circle_center[1] - circle_radius,
                            circle_center[0] + circle_radius, circle_center[1] + circle_radius
                        ), outline="red", width=3)
                        angle_label = "A"
                    else:
                        angle_label = "H"

                    # Calculate text size using textbbox
                    angle_text_bbox = draw.textbbox((0, 0), angle_text, font=font)
                    angle_label_bbox = draw.textbbox((0, 0), angle_label, font=font)

                    angle_text_width = angle_text_bbox[2] - angle_text_bbox[0]
                    angle_text_height = angle_text_bbox[3] - angle_text_bbox[1]
                    angle_label_width = angle_label_bbox[2] - angle_label_bbox[0]
                    angle_label_height = angle_label_bbox[3] - angle_label_bbox[1]

                    # Draw angle value
                    draw.text((point[0], point[1] - 20), angle_text, fill="black", font=font)

                    # Draw angle label
                    label_position = (point[0] + (angle_text_width - angle_label_width) / 2, point[1] + 5)
                    draw.text(label_position, angle_label, fill="black", font=font)

                    draw.line([start_mask_point, end_mask_point], fill="red", width=2)  # mask tangent line
                    if end == 0:  # start point
                        draw.line([point, bone[POINTS_AWAY]], fill="yellow", width=2)  # bone tangent line
                    else:  # end point
                        draw.line([point, bone[-1-POINTS_AWAY]], fill="yellow", width=2)  # bone tangent line

            # Draw image ID in the bottom right corner
            id_text = f"Image ID: {image_name}"
            id_text_bbox = draw.textbbox((0, 0), id_text, font=font)
            id_text_width = id_text_bbox[2] - id_text_bbox[0]
            id_text_height = id_text_bbox[3] - id_text_bbox[1]
            id_position = (image.width - id_text_width - 10, image.height - id_text_height - 10)
            draw.text(id_position, id_text, fill="black", font=font)

            # Save the modified image
            output_path = os.path.join(output_dir, image_name)
            image.save(output_path)

print("Tangent masks saved in:", output_base_dir)

# Get results as red dots for angular bone loss

In [ ]:
!rm -rf angles

In [ ]:
# for checking mandibular or maxillary

def classify_image(keypoints):
    maxillary_count = 0
    mandibular_count = 0

    for keypoint_array in keypoints:
        cej_coords = [keypoint_array[0][1], keypoint_array[1][1]]
        apex_coords = []

        if len(keypoint_array) == 6:
            apex_coords = [keypoint_array[4][1], keypoint_array[5][1]]
        elif len(keypoint_array) == 5:
            apex_coords = [keypoint_array[4][1]]

        cej_above_apex = sum(cej_y < min(apex_coords) for cej_y in cej_coords)
        cej_below_apex = len(cej_coords) - cej_above_apex

        if cej_above_apex >= cej_below_apex:
            mandibular_count += 1
        else:
            maxillary_count += 1

    if maxillary_count >= mandibular_count:
        return "maxillary"
    else:
        return "mandibular"


In [ ]:
# With red dot marking

import os
from PIL import Image, ImageDraw, ImageFont
import math
import numpy as np
import json

# Set constants
LINE_EXTENSION = 40
MASK_SEARCH_DISTANCE = 20
POINTS_AWAY = 1
SKIP_THRESHOLD = 10

font = ImageFont.truetype(r'/usr/share/fonts/truetype/liberation/LiberationMono-Regular.ttf', 25)

# Set the directory paths
images_dir = "images/"
bone_details_dir = "bone_details_denorm"
mask_details_dir = "new_gt_masks/"
keypoints_path = "keypoints/combined_res.json"
output_dir = "angles"

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# get image classifcation to a dictionary
image_classification = {}
with open(keypoints_path, 'r') as f:
    keypoints_data = json.load(f)
for image in keypoints_data:
    image_classification[image['image_id'] + '.jpg'] = classify_image(image['keypoints'])


# Process each file
for filename in os.listdir(bone_details_dir):
    if filename.endswith(".txt"):
        # if filename != "2.txt":
        #     continue
        image_name = os.path.splitext(filename)[0] + ".jpg"
        image_path = os.path.join(images_dir, image_name)
        bone_details_path = os.path.join(bone_details_dir, filename)
        mask_details_path = os.path.join(mask_details_dir, filename)

        # Read bone lines
        bones = []
        with open(bone_details_path, "r") as f:
            bone_details = f.readlines()
            for line in bone_details:
                points = [float(p) for p in line.split(",")]
                points = [(points[i], points[i+1]) for i in range(0, len(points), 2)]
                bones.append(points)

        # Read teeth masks
        masks = []
        with open(mask_details_path, "r") as f:
            mask_details = f.readlines()
            for line in mask_details:
                points = [float(p) for p in line.split()]
                points = [(points[i], points[i+1]) for i in range(0, len(points), 2)]
                masks.append(points)

        if(len(bones) == 0 or len(masks) == 0):
            continue

        # Load the image
        image = Image.open(image_path)
        draw = ImageDraw.Draw(image)

        # Find the tangent points and draw tangent lines
        for bone in bones:
            start_point = bone[0]

            # Find the closest point on masks to the start point of the bone line
            closest_mask_point, closest_mask_index, closest_mask_point_index = min(
                ((p, i, j) for i, mask_points in enumerate(masks) for j, p in enumerate(mask_points)),
                key=lambda t: math.dist(start_point, t[0])
            )

            if math.dist(start_point, closest_mask_point) >= SKIP_THRESHOLD:
                continue

            # Get another point on mask
            nearby_mask_point_index = (closest_mask_point_index + MASK_SEARCH_DISTANCE) % len(masks[closest_mask_index])
            nearby_mask_point = masks[closest_mask_index][nearby_mask_point_index]

            # Ensure that nearby_mask_point is in APEX side
            if image_classification.get(image_name) == "mandibular":
                if nearby_mask_point[1] < start_point[1]:
                    nearby_mask_point_index = (closest_mask_point_index - MASK_SEARCH_DISTANCE) % len(masks[closest_mask_index])
                    nearby_mask_point = masks[closest_mask_index][nearby_mask_point_index]
            if image_classification.get(image_name) == "maxillary":
                if nearby_mask_point[1] > start_point[1]:
                    nearby_mask_point_index = (closest_mask_point_index - MASK_SEARCH_DISTANCE) % len(masks[closest_mask_index])
                    nearby_mask_point = masks[closest_mask_index][nearby_mask_point_index]

            # To extend the lines
            start_mask_point = (
                closest_mask_point[0] - LINE_EXTENSION * (closest_mask_point[0] - nearby_mask_point[0]),
                closest_mask_point[1] - LINE_EXTENSION * (closest_mask_point[1] - nearby_mask_point[1])
            )
            end_mask_point = (
                closest_mask_point[0] + LINE_EXTENSION * (closest_mask_point[0] - nearby_mask_point[0]),
                closest_mask_point[1] + LINE_EXTENSION * (closest_mask_point[1] - nearby_mask_point[1])
            )

            # Calculate vectors for bone and mask tangent lines
            bone_vector = np.array([bone[POINTS_AWAY][0] - start_point[0], bone[POINTS_AWAY][1] - start_point[1]])
            mask_vector = np.array([end_mask_point[0] - start_mask_point[0], end_mask_point[1] - start_mask_point[1]])

            # Calculate dot product and magnitudes
            dot_product = np.dot(bone_vector, mask_vector)
            bone_magnitude = np.linalg.norm(bone_vector)
            mask_magnitude = np.linalg.norm(mask_vector)

            # Calculate angle in radians
            angle_rad = np.arccos(dot_product / (bone_magnitude * mask_magnitude))

            # Convert angle to degrees
            angle_deg = np.degrees(angle_rad)

            # Draw angle text on the image
            angle_text = f"{angle_deg:.2f}°"
            if angle_deg <= 50:
                circle_center = ((start_point[0], start_point[1]))
                circle_radius = 10
                draw.ellipse((
                    circle_center[0] - circle_radius, circle_center[1] - circle_radius,
                    circle_center[0] + circle_radius, circle_center[1] + circle_radius
                ), outline="red", width=3)
                # angle_label = "A"
            else:
                angle_label = "H"

        # Find the tangent points and draw tangent lines
        for bone in bones:
            end_point = bone[-1]

            # Find the closest point on masks to the end point of the bone line
            closest_mask_point, closest_mask_index, closest_mask_point_index = min(
                ((p, i, j) for i, mask_points in enumerate(masks) for j, p in enumerate(mask_points)),
                key=lambda t: math.dist(end_point, t[0])
            )

            if math.dist(end_point, closest_mask_point) >= SKIP_THRESHOLD:
                continue

            # Get another point on mask
            nearby_mask_point_index = (closest_mask_point_index + MASK_SEARCH_DISTANCE) % len(masks[closest_mask_index])
            nearby_mask_point = masks[closest_mask_index][nearby_mask_point_index]

            # Ensure that nearby_mask_point is in APEX side
            if image_classification.get(image_name) == "mandibular":
                if nearby_mask_point[1] < end_point[1]:
                    nearby_mask_point_index = (closest_mask_point_index - MASK_SEARCH_DISTANCE) % len(masks[closest_mask_index])
                    nearby_mask_point = masks[closest_mask_index][nearby_mask_point_index]
            if image_classification.get(image_name) == "maxillary":
                if nearby_mask_point[1] > end_point[1]:
                    nearby_mask_point_index = (closest_mask_point_index - MASK_SEARCH_DISTANCE) % len(masks[closest_mask_index])
                    nearby_mask_point = masks[closest_mask_index][nearby_mask_point_index]

            # To extend the lines
            start_mask_point = (
                closest_mask_point[0] - LINE_EXTENSION * (closest_mask_point[0] - nearby_mask_point[0]),
                closest_mask_point[1] - LINE_EXTENSION * (closest_mask_point[1] - nearby_mask_point[1])
            )
            end_mask_point = (
                closest_mask_point[0] + LINE_EXTENSION * (closest_mask_point[0] - nearby_mask_point[0]),
                closest_mask_point[1] + LINE_EXTENSION * (closest_mask_point[1] - nearby_mask_point[1])
            )

            # Calculate vectors for bone and mask tangent lines
            bone_vector = np.array([bone[-1-POINTS_AWAY][0] - end_point[0], bone[-1-POINTS_AWAY][1] - end_point[1]])
            mask_vector = np.array([end_mask_point[0] - start_mask_point[0], end_mask_point[1] - start_mask_point[1]])

            # Calculate dot product and magnitudes
            dot_product = np.dot(bone_vector, mask_vector)
            bone_magnitude = np.linalg.norm(bone_vector)
            mask_magnitude = np.linalg.norm(mask_vector)

            # Calculate angle in radians
            angle_rad = np.arccos(dot_product / (bone_magnitude * mask_magnitude))

            # Convert angle to degrees
            angle_deg = np.degrees(angle_rad)

            # Draw angle text on the image
            angle_text = f"{angle_deg:.2f}°"
            if angle_deg <= 50:
                circle_center = ((end_point[0], end_point[1]))
                circle_radius = 10
                draw.ellipse((
                    circle_center[0] - circle_radius, circle_center[1] - circle_radius,
                    circle_center[0] + circle_radius, circle_center[1] + circle_radius
                ), width=3, outline="red")
                # angle_label = "A"
            else:
                angle_label = "H"

        # Draw image ID in the bottom right corner
        id_text = f"Image ID: {image_name}"
        id_text_size = draw.textsize(id_text, font=font)
        id_position = (image.width - id_text_size[0] - 10, image.height - id_text_size[1] - 10)
        draw.text(id_position, id_text, fill="black", font=font)

        # Save the modified image
        output_path = os.path.join(output_dir, image_name)
        image.save(output_path)

print("Tangent masks saved in:", output_dir)


# Get gt bone lines for left column of PDF

In [ ]:
# Import necessary libraries
import os
from PIL import Image, ImageDraw

# Define directories
iopa_dir = 'IOPAs'
gt_bone_details_dir = "bone_details"
output_dir = 'iopa_with_gt'

# Create output directory if it does not exist
os.makedirs(output_dir, exist_ok=True)

# Iterate over each image in the iopa_dir
for image_name in os.listdir(iopa_dir):
    if image_name.endswith('.jpg'):
        image_id = os.path.splitext(image_name)[0]
        image_path = os.path.join(iopa_dir, image_name)
        bone_details_path = os.path.join(gt_bone_details_dir, f"{image_id}.txt")
        output_path = os.path.join(output_dir, image_name)

        # Check if bone details file exists
        if os.path.exists(bone_details_path):
            # Open the image
            iopa_img = Image.open(image_path)
            draw = ImageDraw.Draw(iopa_img)

            # Read bone details
            with open(bone_details_path, "r") as f:
                bone_details = f.readlines()

            # Parse and draw bone lines
            for line in bone_details:
                points = [float(p) for p in line.strip().split(",")]
                points = [(points[i], points[i+1]) for i in range(0, len(points), 2)]
                draw.line(points, fill="blue", width=4)

            # Save the image with bone lines
            iopa_img.save(output_path)
            print(f"Processed and saved: {output_path}")
        else:
            print(f"Bone details not found for: {image_name}")

print("Processing completed.")


# Comparison PDF

In [ ]:
!pip install reportlab

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 9.9 MB/s eta 0:00:00


In [ ]:
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from PIL import Image
import os

# Step 3: Define directories
iopa_dir = 'iopa_with_gt'
angles_base_dir = 'angles'
output_dir = 'tanget_pdfs'

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Step 4: Function to create PDF for a specific subdirectory
def create_pdf_for_subdirectory(subdir):
    angles_dir = os.path.join(angles_base_dir, subdir)

    # List all images in the directories
    iopa_images = [f for f in os.listdir(iopa_dir) if f.endswith('.jpg')]
    angles_images = [f for f in os.listdir(angles_dir) if f.endswith('.jpg')]

    # Create a set of IDs from both directories
    iopa_ids = {os.path.splitext(f)[0] for f in iopa_images}
    angles_ids = {os.path.splitext(f)[0] for f in angles_images}

    # Find the common IDs
    common_ids = iopa_ids & angles_ids
    common_ids = sorted([int(id) for id in common_ids])

    # Create a PDF using reportlab
    output_path = os.path.join(output_dir, f'comparison_{subdir}.pdf')
    c = canvas.Canvas(output_path, pagesize=letter)
    width, height = letter

    # Define the maximum number of rows per page
    rows_per_page = 3
    image_margin = 10  # Margin between images and page edges
    vertical_margin = 20  # Margin between rows

    # Calculate the maximum height for each image
    max_image_height = (height - vertical_margin * (rows_per_page + 1)) // rows_per_page
    max_image_width = (width // 2) - (2 * image_margin)

    # Initialize counters
    current_row = 0

    for image_id in common_ids:
        iopa_image_path = os.path.join(iopa_dir, f"{image_id}.jpg")
        angles_image_path = os.path.join(angles_dir, f"{image_id}.jpg")

        # Open images to get their sizes
        iopa_img = Image.open(iopa_image_path)
        angles_img = Image.open(angles_image_path)

        # Resize images to fit the defined max dimensions (keeping aspect ratio)
        iopa_img.thumbnail((max_image_width, max_image_height))
        angles_img.thumbnail((max_image_width, max_image_height))

        # Calculate positions
        iopa_x = image_margin
        iopa_y = height - (current_row + 1) * (max_image_height + vertical_margin)
        angles_x = width // 2 + image_margin
        angles_y = height - (current_row + 1) * (max_image_height + vertical_margin)

        # Draw images on the PDF
        c.drawImage(iopa_image_path, iopa_x, iopa_y, width=iopa_img.width, height=iopa_img.height)
        c.drawImage(angles_image_path, angles_x, angles_y, width=angles_img.width, height=angles_img.height)

        # Add image ID as text
        c.setFont("Helvetica", 10)
        c.drawString(iopa_x, iopa_y - 15, f"Image ID: {image_id}")

        # Increment row counter
        current_row += 1

        # If the maximum number of rows per page is reached, add a new page
        if current_row >= rows_per_page:
            c.showPage()
            current_row = 0

    # Save the PDF
    c.save()

    print(f"PDF created and saved to {output_path}")

# Step 5: Process specific subdirectories
subdirs = ['test', 'train', 'val']
for subdir in subdirs:
    if os.path.isdir(os.path.join(angles_base_dir, subdir)):
        print(f"Processing subdirectory: {subdir}")
        create_pdf_for_subdirectory(subdir)
    else:
        print(f"Subdirectory not found: {subdir}")

print("All PDFs have been created.")

In [ ]:
!cp comparison.pdf comparison_gt_patterns_vs_gt_calculated_patterns.pdf

In [ ]:
!zip -r angular_marked.zip angles/*


# Output for spreadsheet

In [ ]:
!rm -rf angles

In [ ]:
# for checking mandibular or maxillary

def classify_image(keypoints):
    maxillary_count = 0
    mandibular_count = 0

    for keypoint_array in keypoints:
        cej_coords = [keypoint_array[0][1], keypoint_array[1][1]]
        apex_coords = []

        if len(keypoint_array) == 6:
            apex_coords = [keypoint_array[4][1], keypoint_array[5][1]]
        elif len(keypoint_array) == 5:
            apex_coords = [keypoint_array[4][1]]

        cej_above_apex = sum(cej_y < min(apex_coords) for cej_y in cej_coords)
        cej_below_apex = len(cej_coords) - cej_above_apex

        if cej_above_apex >= cej_below_apex:
            mandibular_count += 1
        else:
            maxillary_count += 1

    if maxillary_count >= mandibular_count:
        return "maxillary"
    else:
        return "mandibular"


In [ ]:
import os
import csv
from PIL import Image, ImageDraw, ImageFont
import math
import numpy as np
import json

# Set constants
LINE_EXTENSION = 40
MASK_SEARCH_DISTANCE = 20
POINTS_AWAY = 1
SKIP_THRESHOLD = 20

font = ImageFont.truetype(r'/usr/share/fonts/truetype/liberation/LiberationMono-Regular.ttf', 20)

# Set the directory paths
images_dir = "images/"
bone_details_dir = "bone_details_denorm"
gt_bone_details_dir = "bone_details"
mask_details_dir = "new_gt_masks/"
keypoints_path = "keypoints/combined_res.json"
output_dir = "angles"
csv_output_path = "angles.csv"
ANGULAR_ANGLE = 55

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Get image classification to a dictionary
image_classification = {}
with open(keypoints_path, 'r') as f:
    keypoints_data = json.load(f)
for image in keypoints_data:
    image_classification[image['image_id'] + '.jpg'] = classify_image(image['keypoints'])

# List to collect angle data
angle_data = []

# get all .txt file names in bone_details_dir
# sort them based on number
bone_details_files = [f for f in os.listdir(bone_details_dir) if f.endswith('.txt')]
bone_details_files.sort(key=lambda x: int(x.split('.')[0]))


iopa_dir = 'IOPAs'
iopa_images = [f for f in os.listdir(iopa_dir) if f.endswith('.jpg')]
iopa_ids = {os.path.splitext(f)[0] for f in iopa_images}

# remove from bone_details_files if the id does not include in iopa
bone_details_files = [f for f in bone_details_files if f.split('.')[0] in iopa_ids]
bone_details_files.sort(key=lambda x: int(x.split('.')[0]))

# Process each file
for filename in bone_details_files:
    if filename.endswith(".txt"):
        print(filename)
        # if filename != "1.txt":
        #     continue
        image_name = os.path.splitext(filename)[0] + ".jpg"
        image_path = os.path.join(images_dir, image_name)
        bone_details_path = os.path.join(bone_details_dir, filename)
        mask_details_path = os.path.join(mask_details_dir, filename)

        # Read bone lines
        bones = []
        with open(bone_details_path, "r") as f:
            bone_details = f.readlines()
            for line in bone_details:
                points = [float(p) for p in line.split(",")]
                points = [(points[i], points[i+1]) for i in range(0, len(points), 2)]
                if points[-1][0] < points[0][0]:
                    points.reverse()
                bones.append(points)
        bones.sort(key=lambda bone: bone[0][0])

        # Read teeth masks
        masks = []
        with open(mask_details_path, "r") as f:
            mask_details = f.readlines()
            for line in mask_details:
                points = [float(p) for p in line.split()]
                points = [(points[i], points[i+1]) for i in range(0, len(points), 2)]
                masks.append(points)

        bones = remove_points_inside_masks(bones, masks)

        if len(bones) == 0 or len(masks) == 0:
            continue

        # Load the image
        image = Image.open(image_path)
        draw = ImageDraw.Draw(image)

        # Draw bone lines in blue
        for bone in bones:
            draw.line(bone, fill="blue", width=2)

        # Find the tangent points and draw tangent lines
        for bone in bones:
            for end in [0, -1]:  # start_point is bone[0], end_point is bone[-1]
                point = bone[end]

                # Find the closest point on masks to the point of the bone line
                closest_mask_point, closest_mask_index, closest_mask_point_index = min(
                    ((p, i, j) for i, mask_points in enumerate(masks) for j, p in enumerate(mask_points)),
                    key=lambda t: math.dist(point, t[0])
                )

                if math.dist(point, closest_mask_point) >= SKIP_THRESHOLD:
                    continue

                # Get another point on mask
                nearby_mask_point_index = (closest_mask_point_index + MASK_SEARCH_DISTANCE) % len(masks[closest_mask_index])
                nearby_mask_point = masks[closest_mask_index][nearby_mask_point_index]

                # Ensure that nearby_mask_point is in APEX side
                if image_classification.get(image_name) == "mandibular":
                    if nearby_mask_point[1] < point[1]:
                        nearby_mask_point_index = (closest_mask_point_index - MASK_SEARCH_DISTANCE) % len(masks[closest_mask_index])
                        nearby_mask_point = masks[closest_mask_index][nearby_mask_point_index]
                if image_classification.get(image_name) == "maxillary":
                    if nearby_mask_point[1] > point[1]:
                        nearby_mask_point_index = (closest_mask_point_index - MASK_SEARCH_DISTANCE) % len(masks[closest_mask_index])
                        nearby_mask_point = masks[closest_mask_index][nearby_mask_point_index]

                # To extend the lines
                start_mask_point = (
                    closest_mask_point[0] - LINE_EXTENSION * (closest_mask_point[0] - nearby_mask_point[0]),
                    closest_mask_point[1] - LINE_EXTENSION * (closest_mask_point[1] - nearby_mask_point[1])
                )
                end_mask_point = (
                    closest_mask_point[0] + LINE_EXTENSION * (closest_mask_point[0] - nearby_mask_point[0]),
                    closest_mask_point[1] + LINE_EXTENSION * (closest_mask_point[1] - nearby_mask_point[1])
                )

                # get mid point index of bone
                mid_point_index = len(bone) // 4
                POINTS_AWAY = mid_point_index

                # Calculate vectors for bone and mask tangent lines
                if end == 0:
                    bone_vector = np.array([bone[POINTS_AWAY][0] - point[0], bone[POINTS_AWAY][1] - point[1]])
                else:
                    bone_vector = np.array([bone[-1-POINTS_AWAY][0] - point[0], bone[-1-POINTS_AWAY][1] - point[1]])

                mask_vector = np.array([end_mask_point[0] - start_mask_point[0], end_mask_point[1] - start_mask_point[1]])

                # Calculate dot product and magnitudes
                dot_product = np.dot(bone_vector, mask_vector)
                bone_magnitude = np.linalg.norm(bone_vector)
                mask_magnitude = np.linalg.norm(mask_vector)

                # Calculate angle in radians
                angle_rad = np.arccos(dot_product / (bone_magnitude * mask_magnitude))

                # Convert angle to degrees
                angle_deg = np.degrees(angle_rad)

                # Collect angle data
                angle_data.append([filename.replace(".txt", "") , angle_deg])

                # Draw angle text on the image
                angle_text = f"{angle_deg:.2f}°"
                if end == -1 and angle_deg <= ANGULAR_ANGLE:
                    angle_label = "A"
                else:
                    angle_label = "H"

                # Calculate text size
                angle_text_size = draw.textsize(angle_text, font=font)
                angle_label_size = draw.textsize(angle_label, font=font)

                # Draw angle value
                draw.text((point[0], point[1] - 20), angle_text, fill="black", font=font)

        # Draw image ID in the bottom right corner
        id_text = f"Image ID: {image_name}"
        id_text_size = draw.textsize(id_text, font=font)
        id_position = (image.width - id_text_size[0] - 10, image.height - id_text_size[1] - 10)
        draw.text(id_position, id_text, fill="black", font=font)

        # Save the modified image
        output_path = os.path.join(output_dir, image_name)
        image.save(output_path)

# Write the angle data to a CSV file
with open(csv_output_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['image_id', 'angle'])
    writer.writerows(angle_data)

print("Tangent masks saved in:", output_dir)
print("Angle data saved in:", csv_output_path)
